In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm

from advshotgun import adversary_image
from advshotgun.pretrained_models import Basic_CNN
from advshotgun.attacks.white_box import iFGSM, FGSM, randomize_attack
from torchvision import models, datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

In [2]:
seed = 2023
torch.manual_seed(seed)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print(f"device:{device}")

device:cuda


In [3]:
batch_size = 8

mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

num_classes = mnist_train.classes.__len__()
class_logit = torch.eye(num_classes).cuda()

print(f"number of classess:{num_classes}")
print(f"train size:{mnist_train.__len__()}")
print(f"test size:{mnist_test.__len__()}")

train_dataloader = DataLoader(mnist_train, batch_size = batch_size, shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size = batch_size, shuffle=False)


number of classess:10
train size:60000
test size:10000


In [4]:
model = Basic_CNN.build_Basic_CNN("9920.pth.tar")
model.cuda()
model.eval()

Basic_CNN(
  (conv1_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)

In [7]:
fgsm = FGSM(eps=0.05)
ifgsm = iFGSM(eps=0.05, iteration=3)
benchmark_method = randomize_attack(eps = 0.05)

In [8]:
batch_upper_bound = 1
for batch_idx, (x, gt) in tqdm(enumerate(test_dataloader)):
    if batch_idx == batch_upper_bound:
        break
    x = x.cuda()
    gt = class_logit[gt.cuda()]
    adv = adversary_image(x, gt, batch_size)
    adv.set_attack_method(fgsm)
    adv.set_victim_model(model)
    adv.evaluate()
    
    adv.set_attack_method(benchmark_method)
    adv.evaluate()
    
    adv.set_attack_method(ifgsm)
    adv.evaluate()

1it [00:00, 10.86it/s]

accuracy:1.0
->
accuracy:0.75
L2 norm:1.0760128498077393
accuracy:1.0
->
accuracy:1.0
L2 norm:1.04994797706604
accuracy:1.0
->
accuracy:0.0
L2 norm:2.691500663757324
